In [1]:
import requests
from urllib.parse import urlparse

from GoogleNews import GoogleNews

googlenews = GoogleNews()

In [2]:
def skip_redirect(uri: str) -> str:
    """ Returns destination URI when given redirect URI"""
    #try:
    return requests.get(uri).url
    #except:
    #return "URL Not found!"


In [3]:
def guess_news_source(uri):        
    domain = urlparse(uri).netloc
    return domain.split('.')[-2]

In [29]:
def create_tweet_text(text: str, hashtags: str, url: str):
    cutoff = 280 - len(hashtags) - 5 # for buffer lets leave 5 out
    return text[:cutoff] + "\n" + hashtags + "\n" + url

In [34]:
def get_news(title = "Iran Protests"):
    googlenews = GoogleNews()
    googlenews.set_lang('en')
    #googlenews.set_time_range('12/01/2022','12/02/2022')
    googlenews = GoogleNews(period='4h')
    googlenews.set_encode('utf-8')
    googlenews.get_news('Iran Revolution')

    results = googlenews.results(sort=True)
    if results is None:
        return []
    else:
        return [{
            'title':res['title'],
            'url': "https://" + res['link'],
        } for res in results]

In [36]:
news_title = get_news()[0]['title']
hashtags = "#IranRevolution"

target_news_link = skip_redirect(
    get_news()[0]['url']
)
news_source = guess_news_source(target_news_link).upper()

hashtags = hashtags + " #" + news_source

print(
    create_tweet_text(news_title, hashtags, target_news_link)
    )

Canada lays fresh sanctions against senior Iranian officials, entities - National | Globalnews.ca
#IranRevolution #GLOBALNEWS
https://globalnews.ca/news/9321105/canada-sanctions-iranian-officials-entities-dec/
